ModuleNotFoundError: No module named 'google.colab'

In [40]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image
import ast
import os

from tqdm import tqdm


In [41]:
import pandas as pd

filenames = os.listdir("/projects/sc73/ranzcr-clip-catheter-line-classification/train")




In [42]:
#extracting just the CVC labels
train_data = pd.read_csv("/projects/sc73/ranzcr-clip-catheter-line-classification/train.csv")
train_data = train_data[train_data['CVC - Borderline'] != 1]
train_data['CVC - Present'] = np.where((train_data['CVC - Abnormal'] == 0) & (train_data['CVC - Normal'] == 0) & (train_data['CVC - Borderline'] == 0), 0, 1)

train_data['CVC - Present'].value_counts()

1    20873
0      750
Name: CVC - Present, dtype: int64

In [43]:
sampled_data = pd.concat([train_data[train_data['CVC - Present'] == 1].sample(n=750), train_data[train_data['CVC - Present'] == 0].sample(n=750)])
sampled_data

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,CVC - Present
5125,1.2.826.0.1.3680043.8.498.29798135944425181867...,0,0,0,0,0,0,0,0,0,1,0,60a61bb58,1
18625,1.2.826.0.1.3680043.8.498.56917573254477591662...,0,0,0,0,0,0,0,0,0,1,0,cc44aee1e,1
29064,1.2.826.0.1.3680043.8.498.11749599424730443758...,0,0,1,0,0,0,0,0,0,1,0,252770973,1
25183,1.2.826.0.1.3680043.8.498.84533062276529819766...,1,0,0,0,1,0,0,0,0,1,0,4e9cd7977,1
15555,1.2.826.0.1.3680043.8.498.49761949951848218455...,0,0,0,0,0,0,0,0,0,1,0,036f433dc,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22809,1.2.826.0.1.3680043.8.498.93902612986195069275...,0,0,0,0,0,1,0,0,0,0,0,f34da342b,0
241,1.2.826.0.1.3680043.8.498.73088700281917814818...,0,0,1,0,0,1,0,0,0,0,0,5d44f5f0d,0
812,1.2.826.0.1.3680043.8.498.85658065578219937558...,0,0,1,0,0,0,1,0,0,0,0,1fcdd2b25,0
5106,1.2.826.0.1.3680043.8.498.84390265238997870942...,0,0,1,0,1,0,0,0,0,0,0,cbd3d8a5b,0


In [44]:
studyInstance = sampled_data['StudyInstanceUID'].tolist()
studyInstance

for i in range(len(studyInstance)):
    studyInstance[i] = f'{studyInstance[i]}.jpg'
    
studyInstance   

['1.2.826.0.1.3680043.8.498.29798135944425181867741764889671646820.jpg',
 '1.2.826.0.1.3680043.8.498.56917573254477591662381927833438589539.jpg',
 '1.2.826.0.1.3680043.8.498.11749599424730443758948253678195377134.jpg',
 '1.2.826.0.1.3680043.8.498.84533062276529819766171648406000766083.jpg',
 '1.2.826.0.1.3680043.8.498.49761949951848218455749059001453442446.jpg',
 '1.2.826.0.1.3680043.8.498.40981519393347637453680278096781297052.jpg',
 '1.2.826.0.1.3680043.8.498.82309444747241213141372618882835382602.jpg',
 '1.2.826.0.1.3680043.8.498.31007111818249500067228144131625734696.jpg',
 '1.2.826.0.1.3680043.8.498.45258993479143480195404274416141355225.jpg',
 '1.2.826.0.1.3680043.8.498.81361445628248936097480449911449382182.jpg',
 '1.2.826.0.1.3680043.8.498.56362571928002495216786513544266233241.jpg',
 '1.2.826.0.1.3680043.8.498.11531534547316263713311684295632234083.jpg',
 '1.2.826.0.1.3680043.8.498.15841117153179839120329046729534619540.jpg',
 '1.2.826.0.1.3680043.8.498.10198749153295666130764

In [45]:
def processImages(filenames,train_test):

    imgs = []
    FileIDs = []
    lst = []


    desired_channels = 3

    count = 0
    for file in tqdm(filenames, position=0, leave=True):
        # Making sure the images are the same pixel size
        img = Image.open(f"/projects/sc73/ranzcr-clip-catheter-line-classification/train/{file}")
        img = img.resize((300, 300))
        img = np.array(img)

        # Ensure that the image has 3 channels (RGB)
        if img.shape[-1] != 3:
            img = np.stack((img,) * 3, axis=-1)  # Convert grayscale to RGB by duplicating channels

        imgs.append(img)  # Append the 4D image to the list

        # Save filename for later
        FileIDs.append(file)

        count += 1

    # Convert the list of images to a numpy array
    imgs = np.array(imgs)
    return imgs, FileIDs

In [46]:
train_imgs, fileIDs = processImages(studyInstance, 'train')

100%|██████████| 1500/1500 [01:27<00:00, 17.08it/s]


In [47]:
#cleaning up the labels
UID = []
for i in range(len(fileIDs)):
  UID.append(fileIDs[i].replace(".jpg",""))

UID[0:10]

['1.2.826.0.1.3680043.8.498.29798135944425181867741764889671646820',
 '1.2.826.0.1.3680043.8.498.56917573254477591662381927833438589539',
 '1.2.826.0.1.3680043.8.498.11749599424730443758948253678195377134',
 '1.2.826.0.1.3680043.8.498.84533062276529819766171648406000766083',
 '1.2.826.0.1.3680043.8.498.49761949951848218455749059001453442446',
 '1.2.826.0.1.3680043.8.498.40981519393347637453680278096781297052',
 '1.2.826.0.1.3680043.8.498.82309444747241213141372618882835382602',
 '1.2.826.0.1.3680043.8.498.31007111818249500067228144131625734696',
 '1.2.826.0.1.3680043.8.498.45258993479143480195404274416141355225',
 '1.2.826.0.1.3680043.8.498.81361445628248936097480449911449382182']

In [12]:
columns_to_drop = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Normal',
                  "NGT - Incompletely Imaged", "Swan Ganz Catheter Present", "PatientID", "CVC - Borderline", 'CVC - Abnormal', 'CVC - Normal']

CVCsampled_data = sampled_data.drop(columns=columns_to_drop, axis=1)

CVCsampled_data.set_index('StudyInstanceUID', inplace=True)

In [13]:
CVCsampled_data = CVCsampled_data.reindex(UID)


In [14]:
train_labels = np.empty((0,3))
data = []

for ID in UID:
    data.append(CVCsampled_data.loc[ID])

train_labels_df = pd.DataFrame(data)



train_labels_df = train_labels_df.reindex(UID)
train_labels = train_labels_df['CVC - Present'].values
# train_labels = train_labels_df.values
train_labels

array([1, 1, 1, ..., 0, 0, 0])

In [15]:
from sklearn.model_selection import train_test_split

# Split your data into training and temporary data (combined validation and test)
train_imgs, temp_imgs, train_labels, temp_labels = train_test_split(train_imgs, train_labels, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets
val_imgs, test_imgs, val_labels, test_labels = train_test_split(temp_imgs, temp_labels, test_size=0.5, random_state=42)

In [16]:
print(len(train_labels))
print(len(train_imgs))


1050
1050


In [14]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout, BatchNormalization
# Load the DenseNet121 base model without its top layers
base_model = DenseNet121(weights='imagenet', include_top=False)

# Add dropout and batch normalization to custom fully connected layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [16]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Reduce learning rate when the validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)


# Compile the model with optimizer and loss function
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Save the best model during training
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Model Training with callbacks
history = model.fit(train_imgs, train_labels, epochs=30, validation_data=(val_imgs, val_labels), callbacks=[reduce_lr, early_stop, model_checkpoint])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_imgs, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/30
33/33 [==============================] - 335s 10s/step - loss: 0.8092 - accuracy: 0.6341 - val_loss: 0.6901 - val_accuracy: 0.5200
Epoch 2/30
33/33 [==============================] - 327s 10s/step - loss: 0.5765 - accuracy: 0.7454 - val_loss: 0.6442 - val_accuracy: 0.6089
Epoch 3/30
33/33 [==============================] - 328s 10s/step - loss: 0.4111 - accuracy: 0.8242 - val_loss: 0.5992 - val_accuracy: 0.6889
Epoch 4/30
33/33 [==============================] - 326s 10s/step - loss: 0.1796 - accuracy: 0.9254 - val_loss: 0.7282 - val_accuracy: 0.6222
Epoch 5/30
33/33 [==============================] - 327s 10s/step - loss: 0.1319 - accuracy: 0.9567 - val_loss: 0.6402 - val_accuracy: 0.6622
Epoch 6/30
33/33 [==============================] - 326s 10s/step - loss: 0.0619 - accuracy: 0.9753 - val_loss: 0.6933 - val_accuracy: 0.6978
Epoch 7/30
33/33 [==============================] - 326s 10s/step - loss: 0.0262 - accuracy: 0.9970 - val_loss: 0.7280 - val_accuracy: 0.7244
Epoch 

In [17]:
def buildTrainData(catheter, train_data, size):
    
    train_data = train_data[train_data[f'{catheter} - Present'] == 1]
    train_data = train_data = train_data[~((train_data[f'{catheter} - Normal'] == 1) & (train_data[f'{catheter} - Abnormal'] == 1))]
    train_data[f'{catheter} - Normal/Abnormal'] = np.where(train_data[f'{catheter} - Normal'] == 1, 1, 0)
    sampled_data = pd.concat([train_data[train_data[f'{catheter} - Normal'] == 1].sample(n=size), train_data[train_data[f'{catheter} - Abnormal'] == 1].sample(n=size)])
    
    return sampled_data

In [18]:
sample = buildTrainData('CVC', train_data, 750)
sample


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,CVC - Present,CVC - Normal/Abnormal
25212,1.2.826.0.1.3680043.8.498.11108043051434216161...,0,0,1,0,0,1,0,0,0,1,0,4099d5872,1,1
23056,1.2.826.0.1.3680043.8.498.81914401367896518348...,0,0,0,0,0,0,0,0,0,1,0,386300913,1,1
27015,1.2.826.0.1.3680043.8.498.34415144012741888520...,0,0,0,0,0,0,0,0,0,1,0,0c9913892,1,1
22320,1.2.826.0.1.3680043.8.498.88754391070038246530...,0,0,0,0,0,0,0,0,0,1,0,c34ad2aa3,1,1
13863,1.2.826.0.1.3680043.8.498.44441453615466617228...,0,0,0,0,0,0,0,0,0,1,0,b1463534c,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18281,1.2.826.0.1.3680043.8.498.82030693571162250993...,0,0,0,0,0,0,0,1,0,0,0,6f4d69ec9,1,0
11246,1.2.826.0.1.3680043.8.498.82166619974013732691...,0,0,0,0,0,0,0,1,0,0,0,72129a678,1,0
5834,1.2.826.0.1.3680043.8.498.24492665583305684486...,0,0,0,0,0,0,0,1,0,0,0,55073fece,1,0
24983,1.2.826.0.1.3680043.8.498.95363365709766072176...,0,0,0,0,0,0,0,1,0,0,0,5604cdb53,1,0


In [24]:
def getFileNames(df):
    studyInstance = df['StudyInstanceUID'].tolist()
    studyInstance

    for i in range(len(studyInstance)):
        studyInstance[i] = f'{studyInstance[i]}.jpg'
      
    return studyInstance
    

In [25]:
nAb_data = getFileNames(sample)

cvc_imgs, fileIDs = processImages(nAb_data, 'train')

100%|██████████| 1500/1500 [01:29<00:00, 16.67it/s]


In [26]:
UID = []
for i in range(len(fileIDs)):
  UID.append(fileIDs[i].replace(".jpg",""))


columns_to_drop = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Normal',
                  "NGT - Incompletely Imaged", "Swan Ganz Catheter Present", "PatientID", "CVC - Borderline", 'CVC - Abnormal', 'CVC - Normal', 'CVC - Present']

CVCsample = sample.drop(columns=columns_to_drop, axis=1)

CVCsample.set_index('StudyInstanceUID', inplace=True)

CVCsample


,CVC - Normal/Abnormal
StudyInstanceUID,
1.2.826.0.1.3680043.8.498.11108043051434216161830699489737681751,1
1.2.826.0.1.3680043.8.498.81914401367896518348642216432851373095,1
1.2.826.0.1.3680043.8.498.34415144012741888520360296227836680446,1
1.2.826.0.1.3680043.8.498.88754391070038246530878003342444547273,1
1.2.826.0.1.3680043.8.498.44441453615466617228387474073459217208,1
...,...
1.2.826.0.1.3680043.8.498.82030693571162250993341474603525165996,0
1.2.826.0.1.3680043.8.498.82166619974013732691987468699733186093,0
1.2.826.0.1.3680043.8.498.24492665583305684486011767114054826053,0


In [27]:
train_labels = np.empty((0,3))
data = []

for ID in UID:
    data.append(CVCsample.loc[ID])

train_labels_df = pd.DataFrame(data)



train_labels_df = train_labels_df.reindex(UID)
cvc_labels = train_labels_df['CVC - Normal/Abnormal'].values
# train_labels = train_labels_df.values
cvc_labels

array([1, 1, 1, ..., 0, 0, 0])

In [28]:
from sklearn.model_selection import train_test_split

# Split your data into training and temporary data (combined validation and test)
trainCVC_imgs, temp_imgs, trainCVC_labels, temp_labels = train_test_split(cvc_imgs, cvc_labels, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets
val_imgs, testCVC_imgs, val_labels, testCVC_labels = train_test_split(temp_imgs, temp_labels, test_size=0.5, random_state=42)

In [31]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
base_model = DenseNet121(weights='imagenet', include_top=False)


# Add dropout and batch normalization to custom fully connected layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Reduce learning rate when the validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)


# Compile the model with optimizer and loss function
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Save the best model during training
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Model Training with callbacks
history = model.fit(trainCVC_imgs, trainCVC_labels, epochs=30, validation_data=(val_imgs, val_labels), callbacks=[reduce_lr, early_stop, model_checkpoint])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_imgs, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/30
33/33 [==============================] - 332s 10s/step - loss: 0.9921 - accuracy: 0.4954 - val_loss: 0.7441 - val_accuracy: 0.5111
Epoch 2/30
33/33 [==============================] - 323s 10s/step - loss: 0.7399 - accuracy: 0.6192 - val_loss: 1.0775 - val_accuracy: 0.5156
Epoch 3/30
33/33 [==============================] - 321s 10s/step - loss: 0.6228 - accuracy: 0.6812 - val_loss: 0.9324 - val_accuracy: 0.5156
Epoch 4/30
33/33 [==============================] - 323s 10s/step - loss: 0.4569 - accuracy: 0.7864 - val_loss: 0.8534 - val_accuracy: 0.5378
Epoch 5/30
33/33 [==============================] - 322s 10s/step - loss: 0.3085 - accuracy: 0.8748 - val_loss: 1.2451 - val_accuracy: 0.5378
Epoch 6/30
33/33 [==============================] - 323s 10s/step - loss: 0.1728 - accuracy: 0.9304 - val_loss: 1.1353 - val_accuracy: 0.5244
Epoch 7/30
33/33 [==============================] - 323s 10s/step - loss: 0.0992 - accuracy: 0.9670 - val_loss: 1.1810 - val_accuracy: 0.5244
Epoch 

In [36]:
pip install seaborn


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 292 kB 12.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [48]:


import seanborn as sns

# Make predictions
predictions = model.predict(test_imgs)

# Convert the predictions to a binary array
predictions_binary = np.where(predictions > 0.5, 1, 0)


from sklearn.metrics import confusion_matrix

# Create the confusion matrix
confusion_matrix = confusion_matrix(test_labels, predictions_binary)

plt.gca().patch.set_facecolor('none')



custom_palette = ['#D3E0EA', '#AAB8C4', '#8498A2', '#5D7D88', '#3C6270', 
                  '#1F465E', '#132D4B', '#0A1C36', '#051024', '#020916']

# Create a heatmap plot of the confusion matrix
sns.heatmap(confusion_matrix, annot=True, cmap=custom_palette)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

ModuleNotFoundError: No module named 'seanborn'